In [1]:
import importlib
import pandas
import numpy
import re

In [2]:
from functions.get_mon_data import *
from functions.functions_dataframe import * 
from functions.functions_shared import *
from parameters.filters_lists import *
from parameters.route import *
from parameters.regs import *

In [ ]:
"""
df_raw - база даних завершених закупівель 24 обласних центрів з 1 серпня 2016 до 31 грудня 2019.
В Донецькій області - Краматорськ. В Луганській - Сєвєродонецьк. 
Згерерована у Профейсійному модулі аналітики bipro.prozorro.
"ИдентификаторЛота" є унікальним ідентифікатором рядка.
"""  

In [3]:
df_raw = pandas.read_csv('data/TrainingProZorroForSchoolProject20162019.csv', sep = ';', low_memory = False)
training_data_columns = df_raw.columns
print(df_raw.shape[0])

537893


### Step 0:
#### Створити базу навчальних закладів, які самостійно купують через ProZorro

In [4]:
step_mark = 'step0'
action_mark = 'check'
number_0 = '0'
columns_to_check_step0 = ['Организатор']

# Перший спосіб відібрати навчальні заклади, які є замовниками: пошук за ЄДРПОУ в полі "IDOrganizator"
df_tenders_by_schools_by_edrpou = filter_dataframe(df_raw, positive_edrpous(), ['IDOrganizator'])

# Створити базу для закупівель навчальних закладів, які НЕ є замовниками
df_merged_left = pandas.merge(df_raw, df_tenders_by_schools_by_edrpou, on='ИдентификаторЛота', 
                              how = 'left', indicator = True, suffixes = ('','_temp'))
df_undefined_0 = df_merged_left[df_merged_left._merge=='left_only']
df_undefined_0 = df_undefined_0[training_data_columns]

# Другий спосіб відібрати навчальні заклади, які є замовниками: пошук за ключовими словами в полі "Организатор"
df_step0 = df_undefined_0.copy()

for column in columns_to_check_step0:
    df_step0 = step_check(df_step0, column, number_0)
    
df_tenders_by_schools_by_keywords_0, df_undefined_0 = separate_positives_and_negatives_ogranizator(df_step0, step_mark, action_mark)
print(f"{step_mark}:", "df_tenders_by_schools_by_keywords:", df_tenders_by_schools_by_keywords_0.shape[0])

/home/artem/projects/tmp/school-procurements/functions/functions_dataframe.py:8: UserWarning: This pattern has match groups. To actually get the groups, use str.extract.
  case = False, na = False, regex = True)])


537893 rows. filtered 38817


/home/artem/projects/tmp/school-procurements/functions/functions_dataframe.py:41: UserWarning: This pattern has match groups. To actually get the groups, use str.extract.
  given_df[column + '_step' + number + '_check'] = (given_df[column].str.contains(r'\b({})'.format('|'.join(filter_keywords)), case = False, na = False, regex = True)) |(given_df[column].str.contains(r'\b({})\b'.format('|'.join(filter_keywords_strict)), case = False, na = False, regex = True))


step0: df_tenders_by_schools_by_keywords: 12598


In [5]:
# df_tenders_by_schools_concatenated - 
# Об'єднання двох баз закупівель навчальних закладів, які самостійно купують через ProZorro

frames_tenders_by_schools = [df_tenders_by_schools_by_edrpou, df_tenders_by_schools_by_keywords_0]
df_tenders_by_schools_concatenated = pandas.concat(frames_tenders_by_schools)
len_df = df_tenders_by_schools_concatenated.shape[0]
df_tenders_by_schools_concatenated.drop_duplicates(subset = 'ИдентификаторЛота', inplace = True)
print("Removed duplicates:", len_df - df_tenders_by_schools_concatenated.shape[0])

Removed duplicates: 353


### Step 1: 
#### Створити базу закупівель навчальних закладів, які купують у ProZorro через посередників: управління освіти, міністерства

In [ ]:
# TODO Step 1:
# ПЕРЕВІРИТИ, ЩО ЗНАХОДИТЬ В КОЛОНЦІ "АДЕРСА ПОСТАВКИ"

"""
За результатами точності визначення закупівель вирішити чи фільтрувати за цією колонкою.
Проблема: в полі адреса часто вказують ключові слова, які ніби можуть вказувати на приналежність
закупівлі до шкільних. Наприклад, вул. Шкільна, Гімназійна і т.п.
"""

In [6]:
step_mark = 'step1'
action_mark = 'check'
number_1 = '1'
columns_to_check_step1 = ['Тендер', 'ОписаниеТендера', 'Лот']
df_step1 = df_undefined_0.copy()

for column in columns_to_check_step1:
    df_step1 = step_check(df_step1, column, number_1)

df_tenders_by_schools_by_keywords_1, df_undefined_1 = separate_positives_and_negatives(df_step1, step_mark, action_mark)
df_tenders_by_schools_by_keywords_1 = df_tenders_by_schools_by_keywords_1[training_data_columns]
df_not_school_tenders = df_undefined_1[training_data_columns]
print(f"{step_mark}:", "df_tenders_by_schools_by_keywords_1:", df_tenders_by_schools_by_keywords_1.shape[0])

step1: df_tenders_by_schools_by_keywords_1: 20218


In [ ]:
# TODO:
#    Inspect df_not_school_tenders.csv
#   df_not_school_tenders.to_csv('df_not_school_tenders.csv');

### Step 2:
#### Класифікація навчальних закладів, які самостійно купують через ProZorro

In [26]:
# TODO:
# Визначити пріоритетність фільтрів для визначення категорії. Наприклад, якщо в назві є НВК і дитячий садок.

def classification(given_df, columns_to_classify, filter_list, filter_name):
    for column in columns_to_classify:
        given_df[column + '_' + step_mark + '_' + action_mark] = given_df[column].str.contains('|'.join(filter_list), 
                                                                                               case = False, 
                                                                                               na = False)
    for i in range(given_df.shape[0]):
        for column in columns_to_classify:
            if given_df.loc[given_df.index[i], column + '_' + step_mark + '_classification'].any():
                given_df.loc[given_df.index[i], 'Тип закладу'] = filter_name
                break;
    for column in columns_to_classify:
        given_df = clear_redundant_columns(given_df, step_mark, action_mark, column)
    return given_df

In [25]:
step_mark = 'step2'
action_mark = 'classification'
columns_to_classify_step2 = ['Организатор']
df_step2 = df_tenders_by_schools_concatenated.copy()

In [ ]:
columns_to_classify_step2.append('Тип закладу')

In [28]:
df_step2_random = df_step2.copy().sample(1000)

In [29]:
for idx in range(len(filter_priority)):
    df_step2_random = classification(df_step2_random, columns_to_classify_step2, filter_priority[idx], filter_name_priority[idx])

In [85]:
df_step2_random[df_step2_random['Тип закладу'].notnull()].shape

(690, 12)

In [88]:
print(df_step2_random[df_step2_random['Тип закладу'].isnull()].shape)
df_step2_random[df_step2_random['Тип закладу'].isnull()][columns_to_classify_step2]

(310, 12)


,Тендер,ОписаниеТендера,Лот,Адрес поставки,Тип закладу
318308,Килим,"Килим придвірний вологовбирний для Київської дитячої музичної школи №23 (проспект Л. Курбаса, 10-Є) , розмір 2 м*2,20 м=4,4 кв.м, (1 шт.)\n Вологовбирний килим ""Лас Вегас"" коричн...",Килим,"проспект Л. Курбаса, 10-Є",NaN
515624,Радіатори для господарських потреб СЗШ № 11 та СЗШ № 7,NaN,Радіатори для господарських потреб СЗШ № 11 та СЗШ № 7,"пр.Гвардійський, 25",NaN
452700,Світильники для потреб СЗШ №18,NaN,Світильники для потреб СЗШ №18,"вул. Вілесова, буд. 10",NaN
386296,"Завершальні будівельні роботи (поточний(аварійний ремонт покрівлі теплового вузлу в загально-освітній школі І-ІІІ ступенів № 38 Запорізької міської ради , м.Запоріжжя, вул.Парамонова, 7А","Завершальні будівельні роботи (поточний(аварійний ремонт покрівлі теплового вузлу в загально-освітній школі І-ІІІ ступенів № 38 Запорізької міської ради , м.Запоріжжя, вул.Парамонова, 7А","Завершальні будівельні роботи (поточний(аварійний ремонт покрівлі теплового вузлу в загально-освітній школі І-ІІІ ступенів № 38 Запорізької міської ради , м.Запоріжжя, вул.Парамонова, 7А","Парамонова, 7А",NaN
18784,"Послуги з розробки проектної документації на установку АПС та оповіщення про пожежу в приміщеннях ЗОШ №12 за адресою: м.Миколаїв, вул. 1-а Екіпажна,4",NaN,"Послуги з розробки проектної документації на установку АПС та оповіщення про пожежу в приміщеннях ЗОШ №12 за адресою: м.Миколаїв, вул. 1-а Екіпажна,4",вул.1-а Екіпажна.4,NaN
50939,"Виконання послуги з розробки робочого проекту на капітальний ремонт приміщень №1-10,12 та частини системи опалення будівлі КПСМНЗ ""Дитяча музична школа №14"".","Виконання послуги з розробки робочого проекту на капітальний ремонт приміщень №1-10,12 та частини системи опалення будівлі КПСМНЗ ""Дитяча музична школа №14"" за адресою: м. Харків, провулок 1-й Ліс...","Виконання послуги з розробки робочого проекту на капітальний ремонт приміщень №1-10,12 та частини системи опалення будівлі КПСМНЗ ""Дитяча музична школа №14"".","провулок 1-й Лісопарківський,5",NaN
516278,Послуги з забезпечення функціонування та просування інформаційної платформи «Харківські спортивні шкільні ліги»,Послуги з забезпечення функціонування та просування інформаційної платформи «Харківські спортивні шкільні ліги»,Послуги з забезпечення функціонування та просування інформаційної платформи «Харківські спортивні шкільні ліги»,"майдан Конституції, буд. 7, 1 пов., каб. 11",NaN
179403,"""Капітальний ремонт ЗОШ І-ІІІ ступенів № 13, вул. Євгена Тельнова, 45""","Виконання будівельно-монтажних робіт по об'єкту: ""Капітальний ремонт ЗОШ І-ІІІ ступенів № 13, вул. Євгена Тельнова, 45""","""Капітальний ремонт ЗОШ І-ІІІ ступенів № 13, вул. Євгена Тельнова, 45""","вул. Євгена Тельнова, 45",NaN
87408,ДСТУ Б Д.1.1-1:2013 - Реконструкція спортивного майданчика на території Комунального закладу «Комплексна дитячо-юнацька спортивна школа №5 Харківської міської ради» за адресою: Профспілковий бульв...,NaN,ДСТУ Б Д.1.1-1:2013 - Реконструкція спортивного майданчика на території Комунального закладу «Комплексна дитячо-юнацька спортивна школа №5 Харківської міської ради» за адресою: Профспілковий бульв...,"Профспілковий бульвар, 1",NaN
155403,ДК 021-2015 код 55520000-1 Кейтерингові послуги (Послуги з організації шкільного харчування),NaN,Лот 14,NaN,NaN


In [31]:
df_step2_random[df_step2_random['Тип закладу'] == 'ДНЗ'][columns_to_classify_step2].sample(10)

,Организатор,Тип закладу
108502,"КОМУНАЛЬНИЙ ЗАКЛАД ""ДОШКІЛЬНИЙ НАВЧАЛЬНИЙ ЗАКЛАД №75 ВІННИЦЬКОЇ МІСЬКОЇ РАДИ""",ДНЗ
335936,"ХМЕЛЬНИЦЬКИЙ ДОШКІЛЬНИЙ НАВЧАЛЬНИЙ ЗАКЛАД №20 ""БІЛОЧКА""",ДНЗ
416693,"Сумський дошкільний навчальний заклад (ясла-садок) №2 ""Ясочка м.Суми, Сумської області",ДНЗ
388233,"ХМЕЛЬНИЦЬКИЙ ДОШКІЛЬНИЙ НАВЧАЛЬНИЙ ЗАКЛАД №20 ""БІЛОЧКА""",ДНЗ
157862,Житомирський центр розвитку дитини № 68,ДНЗ
62277,"Сумський дошкільний навчальний заклад (ясла-садок) № 17 ""Радість"" м. Суми, Сумської області",ДНЗ
272106,"КЗО ""НВК"" Вальдорфська середня загальноосвітня школа І-ІІІ ступенів-дитячий садок"" ДМР",ДНЗ
286485,"КОМУНАЛЬНИЙ ЗАКЛАД ""ДОШКІЛЬНИЙ НАВЧАЛЬНИЙ ЗАКЛАД № 13 ВІННИЦЬКОЇ МІСЬКОЇ РАДИ""",ДНЗ
388874,КУ Хмельницький дошкільний навчальний заклад № 53 «Веселка»,ДНЗ
290484,"КОМУНАЛЬНИЙ ЗАКЛАД ""ДОШКІЛЬНИЙ НАВЧАЛЬНИЙ ЗАКЛАД № 43 ВІННИЦЬКОЇ МІСЬКОЇ РАДИ""",ДНЗ


### Step 3:
#### Класифікація навчальних закладів, для яких купують управління освіти, міністерства чи інші органи влади

In [9]:
# TODO:
# ПЕРЕВІРИТИ, ЩО ЗНАХОДИТЬ В КОЛОНЦІ "ОРГАНИЗАТОР"

In [73]:
step_mark = 'step3'
action_mark = 'classification'
columns_to_classify_step3 = ['Тендер', 'ОписаниеТендера', 'Лот', 'Адрес поставки']
df_step3 = df_tenders_by_schools_by_keywords_1.copy()

In [34]:
df_step3_random = df_step3.copy().sample(1000)

In [35]:
for idx in range(len(filter_priority_new)):
    df_step3_random = classification(df_step3_random, columns_to_classify_step3, 
                                    filter_priority[idx], filter_name_priority[idx])

In [ ]:
columns_to_classify_step3.append('Тип закладу')

In [76]:
df_step3_random['Тип закладу'].value_counts()

ДНЗ                                         375
Загальноосвітня школа                       145
Гімназія                                     68
Навчально-виховний комплекс (об'єднання)     49
Спеціалізована школа                         34
Ліцей                                        11
Спеціальна загальноосвітня школа              4
Інтернат                                      4
Name: Тип закладу, dtype: int64

In [85]:
df_step3_random[df_step3_random['Тип закладу'].notnull()].shape

(690, 12)

In [88]:
print(df_step3_random[df_step3_random['Тип закладу'].isnull()].shape)
df_step3_random[df_step3_random['Тип закладу'].isnull()][columns_to_classify_step3]

(310, 12)


,Тендер,ОписаниеТендера,Лот,Адрес поставки,Тип закладу
318308,Килим,"Килим придвірний вологовбирний для Київської дитячої музичної школи №23 (проспект Л. Курбаса, 10-Є) , розмір 2 м*2,20 м=4,4 кв.м, (1 шт.)\n Вологовбирний килим ""Лас Вегас"" коричн...",Килим,"проспект Л. Курбаса, 10-Є",NaN
515624,Радіатори для господарських потреб СЗШ № 11 та СЗШ № 7,NaN,Радіатори для господарських потреб СЗШ № 11 та СЗШ № 7,"пр.Гвардійський, 25",NaN
452700,Світильники для потреб СЗШ №18,NaN,Світильники для потреб СЗШ №18,"вул. Вілесова, буд. 10",NaN
386296,"Завершальні будівельні роботи (поточний(аварійний ремонт покрівлі теплового вузлу в загально-освітній школі І-ІІІ ступенів № 38 Запорізької міської ради , м.Запоріжжя, вул.Парамонова, 7А","Завершальні будівельні роботи (поточний(аварійний ремонт покрівлі теплового вузлу в загально-освітній школі І-ІІІ ступенів № 38 Запорізької міської ради , м.Запоріжжя, вул.Парамонова, 7А","Завершальні будівельні роботи (поточний(аварійний ремонт покрівлі теплового вузлу в загально-освітній школі І-ІІІ ступенів № 38 Запорізької міської ради , м.Запоріжжя, вул.Парамонова, 7А","Парамонова, 7А",NaN
18784,"Послуги з розробки проектної документації на установку АПС та оповіщення про пожежу в приміщеннях ЗОШ №12 за адресою: м.Миколаїв, вул. 1-а Екіпажна,4",NaN,"Послуги з розробки проектної документації на установку АПС та оповіщення про пожежу в приміщеннях ЗОШ №12 за адресою: м.Миколаїв, вул. 1-а Екіпажна,4",вул.1-а Екіпажна.4,NaN
50939,"Виконання послуги з розробки робочого проекту на капітальний ремонт приміщень №1-10,12 та частини системи опалення будівлі КПСМНЗ ""Дитяча музична школа №14"".","Виконання послуги з розробки робочого проекту на капітальний ремонт приміщень №1-10,12 та частини системи опалення будівлі КПСМНЗ ""Дитяча музична школа №14"" за адресою: м. Харків, провулок 1-й Ліс...","Виконання послуги з розробки робочого проекту на капітальний ремонт приміщень №1-10,12 та частини системи опалення будівлі КПСМНЗ ""Дитяча музична школа №14"".","провулок 1-й Лісопарківський,5",NaN
516278,Послуги з забезпечення функціонування та просування інформаційної платформи «Харківські спортивні шкільні ліги»,Послуги з забезпечення функціонування та просування інформаційної платформи «Харківські спортивні шкільні ліги»,Послуги з забезпечення функціонування та просування інформаційної платформи «Харківські спортивні шкільні ліги»,"майдан Конституції, буд. 7, 1 пов., каб. 11",NaN
179403,"""Капітальний ремонт ЗОШ І-ІІІ ступенів № 13, вул. Євгена Тельнова, 45""","Виконання будівельно-монтажних робіт по об'єкту: ""Капітальний ремонт ЗОШ І-ІІІ ступенів № 13, вул. Євгена Тельнова, 45""","""Капітальний ремонт ЗОШ І-ІІІ ступенів № 13, вул. Євгена Тельнова, 45""","вул. Євгена Тельнова, 45",NaN
87408,ДСТУ Б Д.1.1-1:2013 - Реконструкція спортивного майданчика на території Комунального закладу «Комплексна дитячо-юнацька спортивна школа №5 Харківської міської ради» за адресою: Профспілковий бульв...,NaN,ДСТУ Б Д.1.1-1:2013 - Реконструкція спортивного майданчика на території Комунального закладу «Комплексна дитячо-юнацька спортивна школа №5 Харківської міської ради» за адресою: Профспілковий бульв...,"Профспілковий бульвар, 1",NaN
155403,ДК 021-2015 код 55520000-1 Кейтерингові послуги (Послуги з організації шкільного харчування),NaN,Лот 14,NaN,NaN


### Step 4:
####  присвоєння навчальним закладам номерів або власних назв

### Step 5:
####  присвоєння власних назв

### Step 6:
####  зведення в єдину базу

### Optional step

#### За ЄДРПОУ видалити рядки із закупівлями закладів, які не є закладами загальної середньої чи дошкільної освіти

In [ ]:
# TODO: 
# До переліку "негативних" ЄДРПОУ включені управління освіти. 
# Тому заклади, де є "управління освіти" в полі організатор, не видаляти

In [ ]:
# filter_dataframe_inspect_before_drop(df_step1_positives, negative_edrpou(), ['IDOrganizator'])['IDOrganizator'].unique().tolist()

In [ ]:
#filter_dataframe_inspect_before_drop_step_0['Организатор'].unique().tolist()

In [ ]:
# filter_dataframe_inspect_before_drop(df_step1_positives, filter_poza, ['Организатор'])['Организатор'].unique().tolist()

In [10]:
#df_step2_positives = filter_dataframe_drop(df_step1_positives, negative_edrpou(), ['IDOrganizator'])
#df_step2_positives = filter_dataframe_drop(df_step2_positives, filter_poza, ['Организатор'])
#print('Залишилось закупівель навчальних закладів, які не є замовниками:', df_step2_positives.shape[0])